## Lectura y limpieza de Datos Llamadas 123
presentado por: Jorge Enrique Malambo

* cargar y limpiar cada df de los meses 
* Modificar en la tabla de string a date/time 
* Por tabla (de cada mes), puedes hacer un reporte con los datos limpios
* Hacer todo esto en todas las tablas de 123 Y generar la tabla maestra
* hacer reporte de tabal maestra

In [42]:
import os
import sys
import logging  
import pandas as pd

import re # libreria para administracion de expresiones regulares

from pandas_profiling import ProfileReport #para generar reporte


project_dir = os.path.dirname(os.path.abspath('.')) # get the path of the project, if doesn't work use os.getcwd() instead and use rfind to find the project dir
sys.path.append(os.path.join(project_dir, 'src', 'data'))
from data_manipulator import get_input_data

bucket = 'esp_bigdata_jemm'
directory = 'BigData'

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Visualizando el diccionario de datos

In [43]:
file = 'gs://{}/{}/data/metadatos-llamadas-urg-y-emer.csv'.format(bucket, directory)
df_metadata = pd.read_csv(file, encoding='latin1', sep=';')
df_metadata

,NOMBRE,DESCRIPCION
0,FECHA_INCIDENTE,Es la fecha el cual se registra la llamada del...
1,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,Es la fecha el cual se inicia el desplazamient...
2,CODIGO LOCALIDAD,Es el código de las 20 localidades de la ciuda...
3,LOCALIDAD,Es la localidad donde sucede el incidente.
4,EDAD,La edad del paciente.
5,UNIDAD,"La descripción de la edad si es en horas, días..."
6,GENERO,Es la distinción de genero del paciente
7,RED,Es la localización a nivel bogota de la red de...
8,TIPO_INCIDENTE,Es la descripción inicial que tipifica el cent...
9,PRIORIDAD,Es la tipificación según la prioridad del inci...


Para estandarizar los campos de los dataframes, relizare una copia de las variables declaradas en el ***diccionario de datos*** en un arreglo

In [44]:
columName=[x for x in df_metadata['NOMBRE']]
columName

['FECHA_INCIDENTE',
 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL',
 'CODIGO LOCALIDAD',
 'LOCALIDAD',
 'EDAD',
 'UNIDAD',
 'GENERO',
 'RED',
 'TIPO_INCIDENTE',
 'PRIORIDAD',
 'MES',
 'CLASIFICACION FINAL']

Usando la libreria ***re*** y con un recorrido por la lista creada, se realiza un cambio del caracter ***' '*** a ***'_'***, es de anotar que para estas variables no habia una necesidad de quitar los espacios alrededor del strig con el método *strip()*

In [45]:
for index in range(len(columName)):
    columName[index]=re.sub('\s+', '_',columName[index])
columName

['FECHA_INCIDENTE',
 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL',
 'CODIGO_LOCALIDAD',
 'LOCALIDAD',
 'EDAD',
 'UNIDAD',
 'GENERO',
 'RED',
 'TIPO_INCIDENTE',
 'PRIORIDAD',
 'MES',
 'CLASIFICACION_FINAL']

## Ahora pasaremos a los datos crudos:

Inicialemte inicializaré un arreglo con los nombres de los archivos que tengo en el bucket.

In [46]:
dataRaw=['datos-abiertos-julio-2019.csv',
         'datos-abiertos-agosto-2019.csv',
         'datos-abiertos-septiembre-2019.csv',
         'datos-abiertos-noviembre_2019.csv',
         'datos_abiertos_cons_diciembre_2019.csv',
         'datos-abiertos-enero_2020.csv',
         'llamadas-de-urgencias-y-emergencias.csv'
     ]

***Defino un arreglo de dataframes***

In [47]:
dfArrayCalls=list()
for i in range(len(dataRaw)):
    df= get_input_data(bucket=bucket,
                          initial_directory=directory,
                          filename = dataRaw[i])
    dfArrayCalls.append(df)

In [48]:
#una pequeña vista de los dataframes
for i in range(len(dfArrayCalls)):
    print(f'archivo :{i},{dataRaw[i]}\n\n',dfArrayCalls[i].head())

archivo :0,datos-abiertos-julio-2019.csv

        FECHA_INCIDENTE FECHA_INICIO_DESPLAZAMIENTO-MOVIL  CODIGO DE LOCALIDAD  \
0  2019-07-01 00:05:08               2019-07-01 00:11:02                    8   
1  2019-07-01 00:06:57               2019-07-01 00:12:56                   19   
2  2019-07-01 00:09:26               2019-07-01 00:10:26                    6   
3  2019-07-01 00:14:42               2019-07-01 00:15:47                    8   
4  2019-07-01 00:17:13               2019-07-01 00:31:13                   10   

         LOCALIDAD      EDAD    UNIDAD     GENERO    RED  \
0          Kennedy        39      Años   FEMENINO  Norte   
1   Ciudad Bolivar        69      Años  MASCULINO    Sur   
2       Tunjuelito        25      Años  MASCULINO    Sur   
3          Kennedy        65      Años  MASCULINO    Sur   
4         Engativa  SIN_DATO  SIN_DATO   SIN_DATO  Norte   

                           TIPO_INCIDENTE PRIORIDAD    MES CLASIFICACION FINAL  
0                 Dificultad

De esta primera aproximacion de hallan las siguientes inconsistencias:

+ aparicion de  una columna que asigna el numero de incidenteen dataframes:
    + datos-abiertos-agosto-2019.csv
    + datos-abiertos-septiembre-2019.csv
    + datos-abiertos-noviembre_2019.csv
    + datos_abiertos_cons_diciembre_2019.csv



Sin embargo esta columna no esta definida en el ***Diccionario de datos***, debe llamarse FECHA_INCIDENTE. 
    
+ Los datos en ***datos-abiertos-enero_2020.csv*** presenta varias colomnas Unnamed... con elementos vacios, estas tambien se eliminaran sin lios.

Ahora elimimando las UNNAMED...

In [49]:
#generar arreglo de nombres de columnas 5
unnamed=[*range(10,37,1)]
for i in unnamed:
    unnamed[i-10]='Unnamed: '+str(i)
# limpiar columnas Unnamed...    
for nombre in unnamed:
    dfArrayCalls[5]=dfArrayCalls[5].drop([nombre], axis=1)
# ver como queda el dataframe
dfArrayCalls[5]

,NUMERO_INCIDENTE,FECHA_DESPACHO_518,CODIGO DE LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,CRU-00000002-20,2020-01-01 00:13:40,19,Ciudad Bolívar,0,SIN_DATO,SIN_DATO,Sur,Heridos,2
1,CRU-00000002-20,2020-01-01 00:13:40,19,Ciudad Bolívar,0,SIN_DATO,SIN_DATO,Sur,Heridos,2
2,CRU-00000005-20,2020-01-01 00:15:14,14,Los Mártires,0,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,1
3,CRU-00000024-20,2020-01-01 00:20:50,11,Suba,58,Años,MASCULINO,Norte,Inconsciente/Paro Cardiorrespiratorio,1
4,CRU-00000036-20,2020-01-01 00:26:05,10,Engativá,0,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,1
...,...,...,...,...,...,...,...,...,...,...
15299,CRU-00038143-20,2020-02-01 00:03:14,18,Rafael Uribe Uribe,89,Años,FEMENINO,Sur,Síntomas gastrointestinales,3
15300,CRU-00038145-20,2020-02-01 00:03:58,1,Usaquén,0,SIN_DATO,SIN_DATO,Norte,Intento de suicidio,2
15301,CRU-00038109-20,2020-02-01 00:10:45,19,Ciudad Bolívar,46,Años,MASCULINO,Sur,Trastorno mental,3
15302,CRU-00037765-20,2020-02-01 00:55:15,19,Ciudad Bolívar,0,SIN_DATO,SIN_DATO,Sur,Trastorno mental,3


In [50]:
#una pequeña vista de los dataframes
for i in range(len(dfArrayCalls)):
    print(f'archivo :{i},{dataRaw[i]}\n\n',dfArrayCalls[i].head())

archivo :0,datos-abiertos-julio-2019.csv

        FECHA_INCIDENTE FECHA_INICIO_DESPLAZAMIENTO-MOVIL  CODIGO DE LOCALIDAD  \
0  2019-07-01 00:05:08               2019-07-01 00:11:02                    8   
1  2019-07-01 00:06:57               2019-07-01 00:12:56                   19   
2  2019-07-01 00:09:26               2019-07-01 00:10:26                    6   
3  2019-07-01 00:14:42               2019-07-01 00:15:47                    8   
4  2019-07-01 00:17:13               2019-07-01 00:31:13                   10   

         LOCALIDAD      EDAD    UNIDAD     GENERO    RED  \
0          Kennedy        39      Años   FEMENINO  Norte   
1   Ciudad Bolivar        69      Años  MASCULINO    Sur   
2       Tunjuelito        25      Años  MASCULINO    Sur   
3          Kennedy        65      Años  MASCULINO    Sur   
4         Engativa  SIN_DATO  SIN_DATO   SIN_DATO  Norte   

                           TIPO_INCIDENTE PRIORIDAD    MES CLASIFICACION FINAL  
0                 Dificultad

Ahora definire una funcion que renombre los nombres de las columnas de acuerdo a nuestro arreglo tomado de ***Diccionario de datos***

In [56]:
# renombrar los titulos de las columnas de df
def RenombrarColumns(index):
    dfArrayCalls[index].rename(
        columns = {
            dfArrayCalls[index].keys()[0]           : columName[0],
            dfArrayCalls[index].keys()[1]           : columName[1],
            dfArrayCalls[index].keys()[2]           : columName[2],
            dfArrayCalls[index].keys()[3]           : columName[3],
            dfArrayCalls[index].keys()[4]           : columName[4],
            dfArrayCalls[index].keys()[5]           : columName[5],
            dfArrayCalls[index].keys()[6]           : columName[6],
            dfArrayCalls[index].keys()[7]           : columName[7],
            dfArrayCalls[index].keys()[8]           : columName[8],
            dfArrayCalls[index].keys()[9]           : columName[9],
            dfArrayCalls[index].keys()[10]          : columName[10],
            dfArrayCalls[index].keys()[11]          : columName[11]
        },
        inplace=True #esto manipula en memoria cargada para muestra, no el datos en bucket
    )
#dfArrayCalls[0]

Esta fucion se aplicara a los dataframes, pero esta funcion requiere que tengan 12 columnas, razon por la cual se apliacara a los data frames [0,1,6], para las demas la funcion debe redefinirse para no aplicar el scrip a columnas inexistentes.

In [57]:
# renombrar los titulos de las columnas de df
def RenombrarColumns2(index):
    dfArrayCalls[index].rename(
        columns = {
            dfArrayCalls[index].keys()[0]           : columName[0],
            dfArrayCalls[index].keys()[1]           : columName[1],
            dfArrayCalls[index].keys()[2]           : columName[2],
            dfArrayCalls[index].keys()[3]           : columName[3],
            dfArrayCalls[index].keys()[4]           : columName[4],
            dfArrayCalls[index].keys()[5]           : columName[5],
            dfArrayCalls[index].keys()[6]           : columName[6],
            dfArrayCalls[index].keys()[7]           : columName[7],
            dfArrayCalls[index].keys()[8]           : columName[8],
            dfArrayCalls[index].keys()[9]           : columName[9]
        },
        inplace=True #esto manipula en memoria cargada para muestra, no el datos en bucket
    )
#dfArrayCalls[0]

In [58]:
aplicar=[0,1,6]
for ind in aplicar:
    RenombrarColumns(ind)

In [59]:
aplicar=[2,3,4,5]
for ind in aplicar:
    RenombrarColumns2(ind)

In [60]:
#una pequeña vista de los dataframes
for i in range(len(dfArrayCalls)):
    print(f'archivo :{i},{dataRaw[i]}\n\n',dfArrayCalls[i].head())

archivo :0,datos-abiertos-julio-2019.csv

        FECHA_INCIDENTE FECHA_INICIO_DESPLAZAMIENTO_MOVIL  CODIGO_LOCALIDAD  \
0  2019-07-01 00:05:08               2019-07-01 00:11:02                 8   
1  2019-07-01 00:06:57               2019-07-01 00:12:56                19   
2  2019-07-01 00:09:26               2019-07-01 00:10:26                 6   
3  2019-07-01 00:14:42               2019-07-01 00:15:47                 8   
4  2019-07-01 00:17:13               2019-07-01 00:31:13                10   

         LOCALIDAD      EDAD    UNIDAD     GENERO    RED  \
0          Kennedy        39      Años   FEMENINO  Norte   
1   Ciudad Bolivar        69      Años  MASCULINO    Sur   
2       Tunjuelito        25      Años  MASCULINO    Sur   
3          Kennedy        65      Años  MASCULINO    Sur   
4         Engativa  SIN_DATO  SIN_DATO   SIN_DATO  Norte   

                           TIPO_INCIDENTE PRIORIDAD    MES CLASIFICACION_FINAL  
0                 Dificultad Respiratoria     

para esta altura tenemos las dataframes sin columnas fuera del siccionario y con sus columnas con nombre unificados

# Limpiezas de nombres Dataframe

In [61]:
dfArrayCalls[0][columName[3]].value_counts(dropna=False)

 Kennedy               1593
 Suba                  1548
Engativa               1521
 Bosa                  1122
 Ciudad Bolivar         994
Usaquen                 786
San Cristobal           776
 Rafael Uribe Uribe     741
Fontibon                723
 Puente Aranda          690
Usme                    636
Chapinero               599
Barrios Unidos          516
Los Martires            516
Santa Fe                489
Teusaquillo             464
Tunjuelito              462
 Antonio Nariño         292
 La Candelaria           91
 Sumapaz                  1
Name: LOCALIDAD, dtype: int64

In [62]:
#estripieando valores en columna 'LOCADIDAD' en todos los dataframes---pendiente volverlo funcion
for ind in range(len(dfArrayCalls)):
    dfArrayCalls[ind][columName[3]]=dfArrayCalls[ind][columName[3]].apply(lambda x: x.strip())
dfArrayCalls[0][columName[3]].value_counts(dropna=False)

Kennedy               1593
Suba                  1548
Engativa              1521
Bosa                  1122
Ciudad Bolivar         994
Usaquen                786
San Cristobal          776
Rafael Uribe Uribe     741
Fontibon               723
Puente Aranda          690
Usme                   636
Chapinero              599
Barrios Unidos         516
Los Martires           516
Santa Fe               489
Teusaquillo            464
Tunjuelito             462
Antonio Nariño         292
La Candelaria           91
Sumapaz                  1
Name: LOCALIDAD, dtype: int64

In [63]:
#estripeando valores en columan 'GENERO'
for ind in range(len(dfArrayCalls)):
    dfArrayCalls[ind][columName[6]]=dfArrayCalls[ind][columName[6]].apply(lambda x: x.strip())
dfArrayCalls[0][columName[6]].value_counts(dropna=False)

SIN_DATO     6621
MASCULINO    4190
FEMENINO     3749
Name: GENERO, dtype: int64

In [64]:
# limpia y reemplaza en todos los df los valores de la columna 'TIPO_INCIDENTE'
for ind in range(len(dfArrayCalls)):
    dfArrayCalls[ind][columName[8]]=dfArrayCalls[ind][columName[8]].apply(lambda x: x.strip())
dfArrayCalls[0][columName[8]].value_counts(dropna=False)

Accidente de tránsito con heridos/Muertos    4261
Inconsciente/Paro Cardiorrespiratorio        1623
Dificultad Respiratoria                      1398
Enfermo                                      1131
Convulsiones                                 1011
Heridos                                       747
Trastorno mental                              709
Dolor torácico                                611
Lesiones personales                           547
Intento de suicidio                           534
Patología Gineco - obstétrica                 425
Síntomas gastrointestinales                   344
Accidente cerebro vascular                    323
Caída                                         318
Intoxicaciones                                141
Ideas de suicidio                             111
Violencia Sexual                               41
Muerte Natural                                 39
SIN_DATO                                       39
Incendio estructural                           29


ahora para la columna 'CLASIFICACION_FINAL', solo se hara la limpieza para los data frames [0,1,6]

In [66]:
aplicar=[0,1,6]
for ind in aplicar:
    dfArrayCalls[ind][columName[11]].fillna('SIN_DATO', inplace=True)
    dfArrayCalls[ind][columName[11]] = dfArrayCalls[ind][columName[11]].apply(lambda x: x.strip())

In [78]:
ind=6
dfArrayCalls[ind][columName[1]]=pd.to_datetime(dfArrayCalls[ind][columName[1]],format='%Y/%m/%d %H:%M')
dfArrayCalls[ind][columName[1]].value_counts
dfArrayCalls[ind][columName[0]]=pd.to_datetime(dfArrayCalls[ind][columName[1]],format='%Y/%m/%d %H:%M')
dfArrayCalls[ind][columName[0]].value_counts



ValueError: time data 2019-01 01 00:04:37 doesn't match format specified

# generar Informes

In [79]:
nombreInformes=['profile_raw_data_julio-2019.html',
         'profile_raw_data_agosto-2019.html',
         'profile_raw_data_septiembre-2019.html',
         'profile_raw_data_noviembre_2019.html',
         'profile_raw_data_diciembre_2019.html',
         'profile_raw_data_enero_2020.html',
         'profile_raw_data_llamadas-de-urgencias-y-emergencias.html'
     ]

In [80]:
for ind in range(len(dfArrayCalls)):
    profile = ProfileReport(df=dfArrayCalls[ind])
    profile.to_file(output_file=project_dir + '/reports/'+nombreInformes[ind])

Summarize dataset:   0%|          | 0/25 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Summarize dataset:   0%|          | 0/25 [00:00<?, ?it/s]

...

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Summarize dataset:   0%|          | 0/25 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Ahora unire las dataframes en uno solo

In [83]:
dfCallsTotal= pd.concat(dfArrayCalls)

In [86]:
dfCallsTotal.head()

,FECHA_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL
0,2019-07-01 00:11:02,2019-07-01 00:11:02,8,Kennedy,39,Años,FEMENINO,Norte,Dificultad Respiratoria,ALTA,JULIO,Traslado
1,2019-07-01 00:12:56,2019-07-01 00:12:56,19,Ciudad Bolivar,69,Años,MASCULINO,Sur,Inconsciente/Paro Cardiorrespiratorio,ALTA,JULIO,SIN_DATO
2,2019-07-01 00:10:26,2019-07-01 00:10:26,6,Tunjuelito,25,Años,MASCULINO,Sur,Enfermo,MEDIA,JULIO,Traslado
3,2019-07-01 00:15:47,2019-07-01 00:15:47,8,Kennedy,65,Años,MASCULINO,Sur,Lesiones personales,MEDIA,JULIO,Traslado
4,2019-07-01 00:31:13,2019-07-01 00:31:13,10,Engativa,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Enfermo,MEDIA,JULIO,Cancelado


In [88]:
 profile = ProfileReport(df=dfCallsTotal)
profile.to_file(output_file=project_dir + '/reports/'+'profile_raw_data_total.html')

Summarize dataset:   0%|          | 0/26 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
raw_data = get_input_data(bucket=bucket,
                          initial_directory=directory,
                          filename = 'datos-abiertos-julio-2019.csv')
print(raw_data.info())

It seems that this file contains already the fields, but need to rename them

### Rename Columns

In [ ]:
# Don't let white spaces between headers to avoid errors during the pipeline
# This is sometimes called a recipe for etl
raw_data.rename(
    columns = {
        'FECHA_INICIO_DESPLAZAMIENTO-MOVIL' : 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL',
        'CODIGO DE LOCALIDAD'               : 'CODIGO_LOCALIDAD',
        'CLASIFICACION FINAL'               : 'CLASIFICACION_FINAL'
    },
    inplace=True #esto manipula en memoria cargada para muestra, no el datos en bucket
)
raw_data.head()

### Check values per field
--------
1. LOCALIDAD
2. GENERO

In [ ]:
raw_data['LOCALIDAD'].value_counts(dropna=False) #dropna no muestra los nulos en True por defecto

It seems that in some fields exist a white space at the begining of the values, probably at the end too, lets clean them all

In [ ]:
# Let's use the function strip(), this property works for string objects in python that removes whitespaces
raw_data['LOCALIDAD'].apply(
    lambda x: x.strip()        # here we use a lambda function, exclusively for python. Is a one-line function
).value_counts()

In [ ]:
raw_data['LOCALIDAD'] = raw_data['LOCALIDAD'].apply(lambda x: x.strip()) # we assign the output of the function apply to the same column

In [ ]:
raw_data.head()

In [ ]:
col = 'GENERO'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)   # here we could apply the same technique as before to avoid errors

In [ ]:
col = 'TIPO_INCIDENTE'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

In [ ]:
col = 'CLASIFICACION_FINAL'
#raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

Here this columns contains a null value, aka NaN (this is numeric value that is Null), lets replace them first
* NaN --> SIN_DATO

In [ ]:
raw_data[col].fillna('SIN_DATO').value_counts()

In [ ]:
col = 'CLASIFICACION_FINAL'
raw_data[col].fillna('SIN_DATO', inplace=True)
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data.head()

# Profile of the Data
------
first install pandas profiling, from a terminal

````
pip install pandas-profiling
````

In [ ]:
profile = ProfileReport(df=raw_data)
profile.to_file(output_file=project_dir + '/reports/'+'profile_raw_data_julio_2019.html')

Form the initial report of the file we see that are duplicates rows in the table, we need to subtract them

### Remove duplicates

In [ ]:
print('Number of rows before cleaning:', raw_data.shape[0] )
raw_data.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', raw_data.shape[0] )

### Save the final table
use to_csv to save the table

In [ ]:
# saving in a local directory
raw_data.to_csv(project_dir + '/data/processed/' +'clean_llamadas_123_julio_2019.csv', encoding='latin1', sep=';', index=False)

In [ ]:
raw_data

In [ ]:
raw_data_agosto2019

In [ ]:
data = pd.concat([raw_data,raw_data_agosto2019])
data

In [ ]:
pd.concat?